# **Second Project PP (part3)**

Ghadamiyan Lida 407 AI

# Imports

In [1]:
!pip install pymc

#Loading libraries                                                         
import pymc3 as pm
import seaborn as sns
import sklearn
import theano
import theano.tensor as T
import tensorflow as tf 
from tensorflow.keras.optimizers import SGD, Adam, RMSprop 
import os
from tensorflow.keras.models import load_model
from pymc3.theanof import MRG_RandomStreams, set_tt_rng

import numpy as np
import math
import pandas as pd 

from sklearn.preprocessing import scale
from sklearn import datasets
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from warnings import filterwarnings

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

%config InlineBackend.figure_format = 'retina'
floatX = theano.config.floatX
filterwarnings("ignore")
sns.set_style("white") 

! pip install arviz

# Data Set

In [2]:
X, Y = make_moons(noise=0.2, random_state=0, n_samples=1000)
X = scale(X)
X = X.astype(floatX)
Y = Y.astype(floatX)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state = 42)

# Model

In [3]:
# function for building the binary bayesian neural network
def build_bbnn(bbnn_input, bbnn_output, bbnn_input2, bbnn_output2 ,nn_input, nn_output, hidden_layers):

    with pm.Model() as bneural_network:
        bbnn_input = pm.Data("bbnn_input", bbnn_input)
        bbnn_output = pm.Data("bbnn_output", bbnn_output)

        # Prior on weights and biases for the first NN

        # Prior on biases
        biases = pm.Normal('biases', 0, 1, shape = (2, ))

        # Prior on weights from input to the hidden layer
        weights1 = pm.Normal("weights1", 0, 1, shape=(nn_input.shape[1], hidden_layers), 
                            testval = np.random.randn(nn_input.shape[1], hidden_layers)*np.sqrt(2/hidden_layers).astype(floatX))

        # Prior on weights from the first to the second hidden layer
        weights2 = pm.Normal("weights2", 0, 1, shape=(hidden_layers,), 
                            testval = np.random.randn(hidden_layers)*np.sqrt(2/hidden_layers).astype(floatX))

        # Hyperbolic tangent as activation function
        activation1 = pm.math.tanh(pm.math.dot(bbnn_input, weights1) + biases[0])
        activationOut = pm.math.sigmoid(pm.math.dot(activation1, weights2) + biases[1])

        #############################################################################################################################################
        bbnn_input2 = pm.Data("bbnn_input2", bbnn_input2)
        bbnn_output2 = pm.Data("bbnn_output2", bbnn_output2)


        # Prior on weights and biases for the second NN

        # Prior on biases
        biases2 = pm.Normal('biases2', 0, 1, shape = (2, ))

        # Prior on weights from input to the hidden layer
        weights12 = pm.Normal("weights12", 0, 1, shape=(nn_input.shape[1], hidden_layers), 
                            testval = np.random.randn(nn_input.shape[1], hidden_layers)*np.sqrt(2/hidden_layers).astype(floatX))

        # Prior on weights from the first to the second hidden layer
        weights22 = pm.Normal("weights22", 0, 1, shape=(hidden_layers,), 
                            testval = np.random.randn(hidden_layers)*np.sqrt(2/hidden_layers).astype(floatX))

        # Hyperbolic tangent as activation function
        activation12 = pm.math.tanh(pm.math.dot(bbnn_input2, weights12) + biases2[0])
        activationOut2 = pm.math.sigmoid(pm.math.dot(activation12, weights22) + biases2[1])

        ###############################################################################################################################################
        unif = pm.Normal('unif', 0, 1, total_size = len(nn_output))


        activationUnif = pm.math.sigmoid(pm.math.dot((activationOut2+activationOut), unif))

        
        output = pm.Bernoulli('output', activationUnif, observed = bbnn_output2, total_size = len(nn_output))
    return bneural_network


In [4]:
bneural_network = build_bbnn(X_train, Y_train, X_train, Y_train, X_train, Y_train , 10)
set_tt_rng(MRG_RandomStreams(42))

with bneural_network:
    inference = pm.ADVI()
    approx = pm.fit(n=5000, method=inference)
trace = approx.sample(draws=100)

x = T.matrix("X")
n = T.iscalar("n")

x.tag.test_value = np.empty_like(X_test[:10])
n.tag.test_value = 100
_sample_proba = approx.sample_node(
    bneural_network.output.distribution.p, size=n, more_replacements={bneural_network["bbnn_input"]: x}
)
sample_proba = theano.function([x, n], _sample_proba, on_unused_input='ignore')
pred = sample_proba(X_test, 1000).mean(0) > 0.5

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
Average Loss = 1.7144: 100%|██████████| 5000/5000 [00:07<00:00, 705.99it/s]
Finished [100%]: Average Loss = 1.7139


In [5]:
print(accuracy_score(Y_test, pred))

0.53
